In [ ]:
# DEFINE DATASET !!!!!!!!!!! 
# prev and next

In [157]:
import re
import math
import warnings
warnings.filterwarnings("ignore")
import datetime
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [158]:
pd.set_option('display.max_columns', 550)
pd.set_option('display.max_rows', 550)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [159]:
# no students
mask_mar_apr_ns = pd.read_csv("C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/higher_bundles_mar_apr.csv", sep='|', parse_dates= ["DAY_MONTH"], dayfirst=True)
mask_may_ns = pd.read_csv("C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/higher_bundles_may.csv", sep='|', parse_dates= ["DAY_MONTH"], dayfirst=True)
# students
mask_mar_students = pd.read_csv("C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/Y_students_MAR22.csv", sep='|', parse_dates= ["DAY_MONTH"], dayfirst=True)
mask_apr_students = pd.read_csv("C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/Y_students_APR22.csv", sep='|', parse_dates= ["DAY_MONTH"], dayfirst=True)
mask_may_students = pd.read_csv("C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/Y_students_MAY22.csv", sep='|', parse_dates= ["DAY_MONTH"], dayfirst=True)

In [160]:
mask_mar_ns = mask_mar_apr_ns[(mask_mar_apr_ns["DAY_MONTH"] >= '2022-03-01') & (mask_mar_apr_ns["DAY_MONTH"] <= '2022-03-31')]
mask_apr_ns = mask_mar_apr_ns[(mask_mar_apr_ns["DAY_MONTH"] >= '2022-04-01') & (mask_mar_apr_ns["DAY_MONTH"] <= '2022-04-30')]

In [161]:
mask_mar_ns.head()

MONTH  DAY_MONTH      MSISDN  \
0  01/03/2022 00:00:00 2022-03-01  6906269227   
1  01/03/2022 00:00:00 2022-03-01  6906423181   
2  01/03/2022 00:00:00 2022-03-01  6906454111   
3  01/03/2022 00:00:00 2022-03-01  6906604729   
4  01/03/2022 00:00:00 2022-03-01  6906663027   

                                      BUNDLE_NAME BUNDLE_GROUP TP_CATEGORY  \
0  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
1  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
2  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
3  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
4  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   

   NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE REQUESTING_SYSTEM  
0                    1            10.0              CUapp             CUapp  
1                    1            10.0              RECUR               IVR  
2                    1            10.0             CUsite            CUsite  
3                    1            10.0              CUapp             CUapp  
4                    1            10.0                IVR               IVR

# APPEND TWO Y DATASETS

# MARCH

In [162]:
mask_mar_ns.shape
mask_mar_students.shape

(437973, 10)

(191954, 3)

In [163]:
# Append two y datasets and drop duplicates
mask_mar = mask_mar_ns.append(other=mask_mar_students, ignore_index=True)
mask_mar.shape

(629927, 10)

In [164]:
mask_mar[mask_mar["MSISDN"].duplicated()].shape

(55540, 10)

# APRIL

In [165]:
mask_apr_ns.shape
mask_apr_students.shape

(411949, 10)

(185874, 3)

In [166]:
# Append two y datasets and drop duplicates
mask_apr = mask_apr_ns.append(other=mask_apr_students, ignore_index=True)
mask_apr.shape

(597823, 10)

In [167]:
mask_apr[mask_apr["MSISDN"].duplicated()].shape

(35368, 10)

# MAY

In [168]:
mask_may_ns.shape
mask_may_students.shape

(414296, 10)

(194589, 3)

In [169]:
# Append two y datasets and drop duplicates
mask_may = mask_may_ns.append(other=mask_may_students, ignore_index=True)
mask_may.shape

(608885, 10)

In [170]:
mask_may[mask_may["MSISDN"].duplicated()].shape

(42521, 10)

# DEFINE MY DATASET FOR PROCESSING

In [171]:
# MARCH
mask_prev = mask_mar
mask_next = mask_apr

out_dir = 'C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/Y_MAR_APR_22.csv'

# APRIL
# mask_prev = mask_apr
# mask_next = mask_may

# out_dir = 'C:/Users/zagkoui/OneDrive - Vodafone Group/Documents/HIGHER_BUNDLES/database_y/Y_APR_MAY_22.csv'

In [172]:
mask_prev["BUNDLE_NAME"].value_counts()

Voice/SMS Bundle CU Combo 500+100 Subscription        375141
Combo Bundle CU Student Xclusive 7 Subscription       134460
Combo Bundle CU 10E Subscription                       49288
Combo Bundle CU Student Xclusive 10 Subscription       35539
Combo Bundle CU Student Xclusive 12 Subscription       20839
Combo Bundle CU 700+700+700MB Subscription             13109
Combo Bundle CU Student Xclusive 17.5 Subscription      1116
Combo Bundle CU 13E Subscription                         435
Name: BUNDLE_NAME, dtype: int64

In [173]:
mask_next["BUNDLE_NAME"].value_counts()

Voice/SMS Bundle CU Combo 500+100 Subscription        323914
Combo Bundle CU Student Xclusive 7 Subscription       130300
Combo Bundle CU 10E Subscription                       40112
Combo Bundle CU Student Xclusive 10 Subscription       36045
Combo Bundle CU 13E Subscription                       28946
Combo Bundle CU 700+700+700MB Subscription             18977
Combo Bundle CU Student Xclusive 12 Subscription       17855
Combo Bundle CU Student Xclusive 17.5 Subscription      1674
Name: BUNDLE_NAME, dtype: int64

# ADD PRICE COLUMN

In [174]:
# create a list of our conditions
conditions = [
    ((mask_prev["BUNDLE_NAME"] == 'Voice/SMS Bundle CU Combo 500+100 Subscription') | (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 10 Subscription')),
    ((mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU 10E Subscription') | (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 12 Subscription')),
    (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU 13E Subscription'),
    (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU 700+700+700MB Subscription'),
    (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 7 Subscription'),
    (mask_prev["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 17.5 Subscription')
    ]

# create a list of the values we want to assign for each condition
values = [10, 12, 13.5, 15, 8.5, 17.5]

# create a new column and use np.select to assign values to it using our lists as arguments
mask_prev['BUNDLE_PRICE'] = np.select(conditions, values)


# display updated DataFrame
mask_prev.head()

MONTH  DAY_MONTH      MSISDN  \
0  01/03/2022 00:00:00 2022-03-01  6906269227   
1  01/03/2022 00:00:00 2022-03-01  6906423181   
2  01/03/2022 00:00:00 2022-03-01  6906454111   
3  01/03/2022 00:00:00 2022-03-01  6906604729   
4  01/03/2022 00:00:00 2022-03-01  6906663027   

                                      BUNDLE_NAME BUNDLE_GROUP TP_CATEGORY  \
0  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
1  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
2  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
3  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
4  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   

   NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE REQUESTING_SYSTEM  \
0                  1.0            10.0              CUapp             CUapp   
1                  1.0            10.0              RECUR               IVR   
2                  1.0            10.0             CUsite            CUsite   
3                  1.0            10.0              CUapp             CUapp   
4                  1.0            10.0                IVR               IVR   

   BUNDLE_PRICE  
0          10.0  
1          10.0  
2          10.0  
3          10.0  
4          10.0

In [175]:
# create a list of our conditions
conditions = [
    ((mask_next["BUNDLE_NAME"] == 'Voice/SMS Bundle CU Combo 500+100 Subscription') | (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 10 Subscription')),
    ((mask_next["BUNDLE_NAME"] == 'Combo Bundle CU 10E Subscription') | (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 12 Subscription')),
    (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU 13E Subscription'),
    (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU 700+700+700MB Subscription'),
    (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 7 Subscription'),
    (mask_next["BUNDLE_NAME"] == 'Combo Bundle CU Student Xclusive 17.5 Subscription')
    ]

# create a list of the values we want to assign for each condition
values = [10, 12, 13.5, 15, 8.5, 17.5]

# create a new column and use np.select to assign values to it using our lists as arguments
mask_next['BUNDLE_PRICE'] = np.select(conditions, values)


# display updated DataFrame
mask_next.head()

MONTH  DAY_MONTH      MSISDN  \
0  01/04/2022 00:00:00 2022-04-01  6907651352   
1  01/04/2022 00:00:00 2022-04-01  6907914443   
2  01/04/2022 00:00:00 2022-04-01  6908262003   
3  01/04/2022 00:00:00 2022-04-01  6906151556   
4  01/04/2022 00:00:00 2022-04-01  6943717431   

                                      BUNDLE_NAME BUNDLE_GROUP TP_CATEGORY  \
0                Combo Bundle CU 10E Subscription     CU COMBO          CU   
1  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
2                Combo Bundle CU 10E Subscription     CU COMBO          CU   
3  Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO          CU   
4                Combo Bundle CU 13E Subscription     CU COMBO          CU   

   NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE REQUESTING_SYSTEM  \
0                  1.0            12.0              RECUR               VOP   
1                  1.0            10.0              CUapp             CUapp   
2                  1.0            12.0              VF489            VFShop   
3                  1.0            10.0              CUapp             CUapp   
4                  1.0            13.5                IVR               IVR   

   BUNDLE_PRICE  
0          12.0  
1          10.0  
2          12.0  
3          10.0  
4          13.5

# DUPLICATES

# PREVIOUS

In [176]:
mask_prev[mask_prev["MSISDN"].duplicated(keep=False)].shape

(103628, 11)

In [177]:
mask_prev = mask_prev.sort_values(by=['MSISDN'])

In [178]:
prev_duplicates = mask_prev[mask_prev["MSISDN"].duplicated(keep=False)]
prev_duplicates.head()

MONTH  DAY_MONTH      MSISDN  \
206902  01/03/2022 00:00:00 2022-03-05  6906006344   
42785   01/03/2022 00:00:00 2022-03-19  6906006344   
504030                  NaN 2022-03-01  6906053943   
602253                  NaN 2022-03-29  6906053943   
417245  01/03/2022 00:00:00 2022-03-24  6906057102   

                                            BUNDLE_NAME BUNDLE_GROUP  \
206902   Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO   
42785    Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO   
504030  Combo Bundle CU Student Xclusive 7 Subscription          NaN   
602253  Combo Bundle CU Student Xclusive 7 Subscription          NaN   
417245   Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO   

       TP_CATEGORY  NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE  \
206902          CU                  1.0            10.0              VE284   
42785           CU                  1.0            10.0              VE284   
504030         NaN                  NaN             NaN                NaN   
602253         NaN                  NaN             NaN                NaN   
417245          CU                  1.0            10.0              CUapp   

       REQUESTING_SYSTEM  BUNDLE_PRICE  
206902            VFShop          10.0  
42785             VFShop          10.0  
504030               NaN           8.5  
602253               NaN           8.5  
417245             CUapp          10.0

In [179]:
# count how many bundles purchased customers with more than 1
prev_duplicates.groupby(by=["MSISDN"]).size().value_counts()

2     42686
3      4103
4       881
5       251
6        90
7        39
8        19
9         7
11        5
10        4
17        1
16        1
12        1
dtype: int64

# Keep the higher bundle that purchased

In [180]:
mask_prev.sort_values(by=['BUNDLE_PRICE'], ascending=False)

MONTH  DAY_MONTH      MSISDN  \
482507   NaN 2022-03-13  6940624999   
560643   NaN 2022-03-27  6970838096   
618137   NaN 2022-03-15  6970984522   
560696   NaN 2022-03-13  6942528833   
553131   NaN 2022-03-17  6948685420   
...      ...        ...         ...   
477280   NaN 2022-03-06  6955854688   
549077   NaN 2022-03-10  6947215939   
562458   NaN 2022-03-11  6955854675   
523609   NaN 2022-03-19  6942281903   
480615   NaN 2022-03-03  6946249473   

                                              BUNDLE_NAME BUNDLE_GROUP  \
482507  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
560643  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
618137  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
560696  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
553131  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
...                                                   ...          ...   
477280    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
549077    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
562458    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
523609    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
480615    Combo Bundle CU Student Xclusive 7 Subscription          NaN   

       TP_CATEGORY  NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE  \
482507         NaN                  NaN             NaN                NaN   
560643         NaN                  NaN             NaN                NaN   
618137         NaN                  NaN             NaN                NaN   
560696         NaN                  NaN             NaN                NaN   
553131         NaN                  NaN             NaN                NaN   
...            ...                  ...             ...                ...   
477280         NaN                  NaN             NaN                NaN   
549077         NaN                  NaN             NaN                NaN   
562458         NaN                  NaN             NaN                NaN   
523609         NaN                  NaN             NaN                NaN   
480615         NaN                  NaN             NaN                NaN   

       REQUESTING_SYSTEM  BUNDLE_PRICE  
482507               NaN          17.5  
560643               NaN          17.5  
618137               NaN          17.5  
560696               NaN          17.5  
553131               NaN          17.5  
...                  ...           ...  
477280               NaN           8.5  
549077               NaN           8.5  
562458               NaN           8.5  
523609               NaN           8.5  
480615               NaN           8.5  

[629927 rows x 11 columns]

In [181]:
mask_prev = mask_prev.drop_duplicates(subset="MSISDN", keep="first")

In [182]:
mask_prev[mask_prev["MSISDN"].duplicated(keep=False)].shape

(0, 11)

In [183]:
mask_prev["BUNDLE_PRICE"].value_counts()

10.0    372026
8.5     127480
12.0     63394
15.0     10212
17.5       933
13.5       342
Name: BUNDLE_PRICE, dtype: int64

# NEXT

In [184]:
mask_next[mask_next["MSISDN"].duplicated(keep=False)].shape

(66552, 11)

In [185]:
mask_next = mask_next.sort_values(by=['MSISDN'])

In [186]:
next_duplicates = mask_next[mask_next["MSISDN"].duplicated(keep=False)]
next_duplicates.head()

MONTH  DAY_MONTH      MSISDN  \
251110  01/04/2022 00:00:00 2022-04-08  6900036262   
392732  01/04/2022 00:00:00 2022-04-24  6900036262   
280584  01/04/2022 00:00:00 2022-04-11  6906057102   
226676  01/04/2022 00:00:00 2022-04-05  6906057102   
505074                  NaN 2022-04-29  6906064957   

                                             BUNDLE_NAME BUNDLE_GROUP  \
251110                  Combo Bundle CU 10E Subscription     CU COMBO   
392732                  Combo Bundle CU 13E Subscription     CU COMBO   
280584        Combo Bundle CU 700+700+700MB Subscription     CU COMBO   
226676    Voice/SMS Bundle CU Combo 500+100 Subscription     CU COMBO   
505074  Combo Bundle CU Student Xclusive 12 Subscription          NaN   

       TP_CATEGORY  NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE  \
251110          CU                  1.0            12.0              CUapp   
392732          CU                  1.0            13.5              CUapp   
280584          CU                  1.0            15.0              CUapp   
226676          CU                  1.0            10.0              CUapp   
505074         NaN                  NaN             NaN                NaN   

       REQUESTING_SYSTEM  BUNDLE_PRICE  
251110             CUapp          12.0  
392732             CUapp          13.5  
280584             CUapp          15.0  
226676             CUapp          10.0  
505074               NaN          12.0

In [187]:
# count how many bundles purchased customers with more than 1
next_duplicates.groupby(by=["MSISDN"]).size().value_counts()

2     28050
3      2443
4       481
5       131
6        48
7        13
8         9
10        3
9         3
11        2
14        1
dtype: int64

# Keep the higher bundle that purchased

In [188]:
mask_next.sort_values(by=['BUNDLE_PRICE'], ascending=False)

MONTH  DAY_MONTH      MSISDN  \
415675   NaN 2022-04-06  6946485969   
556374   NaN 2022-04-27  6976884378   
556809   NaN 2022-04-28  6942058893   
548931   NaN 2022-04-21  6955996803   
430548   NaN 2022-04-28  6949308776   
...      ...        ...         ...   
486410   NaN 2022-04-06  6943780990   
432242   NaN 2022-04-23  6949235835   
429469   NaN 2022-04-20  6949235848   
534962   NaN 2022-04-29  6949235850   
432234   NaN 2022-04-23  6946510764   

                                              BUNDLE_NAME BUNDLE_GROUP  \
415675  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
556374  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
556809  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
548931  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
430548  Combo Bundle CU Student Xclusive 17.5 Subscrip...          NaN   
...                                                   ...          ...   
486410    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
432242    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
429469    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
534962    Combo Bundle CU Student Xclusive 7 Subscription          NaN   
432234    Combo Bundle CU Student Xclusive 7 Subscription          NaN   

       TP_CATEGORY  NO_OF_SUBSCRIPTIONS  BUNDLE_REVENUE ENHANCED_REFERENCE  \
415675         NaN                  NaN             NaN                NaN   
556374         NaN                  NaN             NaN                NaN   
556809         NaN                  NaN             NaN                NaN   
548931         NaN                  NaN             NaN                NaN   
430548         NaN                  NaN             NaN                NaN   
...            ...                  ...             ...                ...   
486410         NaN                  NaN             NaN                NaN   
432242         NaN                  NaN             NaN                NaN   
429469         NaN                  NaN             NaN                NaN   
534962         NaN                  NaN             NaN                NaN   
432234         NaN                  NaN             NaN                NaN   

       REQUESTING_SYSTEM  BUNDLE_PRICE  
415675               NaN          17.5  
556374               NaN          17.5  
556809               NaN          17.5  
548931               NaN          17.5  
430548               NaN          17.5  
...                  ...           ...  
486410               NaN           8.5  
432242               NaN           8.5  
429469               NaN           8.5  
534962               NaN           8.5  
432234               NaN           8.5  

[597823 rows x 11 columns]

In [189]:
mask_next = mask_next.drop_duplicates(subset="MSISDN", keep="first")

In [190]:
mask_next[mask_next["MSISDN"].duplicated(keep=False)].shape

(0, 11)

In [191]:
mask_next["BUNDLE_PRICE"].value_counts()

10.0    335300
8.5     126035
12.0     54286
13.5     27554
15.0     17742
17.5      1538
Name: BUNDLE_PRICE, dtype: int64

# MERGE PREV - NEXT !!!!!!!!!!!!!!!!!!!!

#  THA KANW LEFT JOIN GIATI THELW NA KRATHSW PERIPTWSEIS POU TON ENA MHN EVALE KAI TON ALLO DEN EVALE

In [192]:
mask_prev.shape
mask_next.shape

(574387, 11)

(562455, 11)

In [193]:
bundles_summary = mask_prev.merge(mask_next, how='left', on='MSISDN')
bundles_summary.head(4)

MONTH_x DAY_MONTH_x      MSISDN  \
0  01/03/2022 00:00:00  2022-03-07  6900030200   
1  01/03/2022 00:00:00  2022-03-17  6900030559   
2                  NaN  2022-03-24  6900034000   
3  01/03/2022 00:00:00  2022-03-07  6900036262   

                                     BUNDLE_NAME_x BUNDLE_GROUP_x  \
0   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
1   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            NaN   
3   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   

  TP_CATEGORY_x  NO_OF_SUBSCRIPTIONS_x  BUNDLE_REVENUE_x ENHANCED_REFERENCE_x  \
0            CU                    1.0              10.0                CUapp   
1            CU                    1.0              10.0                CUapp   
2           NaN                    NaN               NaN                  NaN   
3            CU                    1.0              10.0                CUapp   

  REQUESTING_SYSTEM_x  BUNDLE_PRICE_x              MONTH_y DAY_MONTH_y  \
0               CUapp            10.0  01/04/2022 00:00:00  2022-04-06   
1               CUapp            10.0  01/04/2022 00:00:00  2022-04-18   
2                 NaN             8.5                  NaN  2022-04-20   
3               CUapp            10.0  01/04/2022 00:00:00  2022-04-08   

                                     BUNDLE_NAME_y BUNDLE_GROUP_y  \
0                 Combo Bundle CU 10E Subscription       CU COMBO   
1                 Combo Bundle CU 13E Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            NaN   
3                 Combo Bundle CU 10E Subscription       CU COMBO   

  TP_CATEGORY_y  NO_OF_SUBSCRIPTIONS_y  BUNDLE_REVENUE_y ENHANCED_REFERENCE_y  \
0            CU                    1.0              12.0                CUapp   
1            CU                    1.0              13.5                CUapp   
2           NaN                    NaN               NaN                  NaN   
3            CU                    1.0              12.0                CUapp   

  REQUESTING_SYSTEM_y  BUNDLE_PRICE_y  
0               CUapp            12.0  
1               CUapp            13.5  
2                 NaN             8.5  
3               CUapp            12.0

In [194]:
# Fill NaNs
for column in bundles_summary.columns.values:
    if (bundles_summary[column].dtype == 'int64' or bundles_summary[column].dtype == 'float64'):
        bundles_summary[column] = bundles_summary[column].fillna(0)
    elif (bundles_summary[column].dtype == 'object'):
        bundles_summary[column] = bundles_summary[column].fillna("N/A")
    elif (bundles_summary[column].dtype == 'datetime64[ns]'):
        new_date =  dt.datetime(int(bundles_summary[column].dt.year.mean()), int(bundles_summary[column].dt.month.mean()) , int(bundles_summary[column].dt.day.mean()))
        bundles_summary[column] = bundles_summary[column].fillna(new_date)

In [195]:
bundles_summary.head(4)

MONTH_x DAY_MONTH_x      MSISDN  \
0  01/03/2022 00:00:00  2022-03-07  6900030200   
1  01/03/2022 00:00:00  2022-03-17  6900030559   
2                  N/A  2022-03-24  6900034000   
3  01/03/2022 00:00:00  2022-03-07  6900036262   

                                     BUNDLE_NAME_x BUNDLE_GROUP_x  \
0   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
1   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            N/A   
3   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   

  TP_CATEGORY_x  NO_OF_SUBSCRIPTIONS_x  BUNDLE_REVENUE_x ENHANCED_REFERENCE_x  \
0            CU                    1.0              10.0                CUapp   
1            CU                    1.0              10.0                CUapp   
2           N/A                    0.0               0.0                  N/A   
3            CU                    1.0              10.0                CUapp   

  REQUESTING_SYSTEM_x  BUNDLE_PRICE_x              MONTH_y DAY_MONTH_y  \
0               CUapp            10.0  01/04/2022 00:00:00  2022-04-06   
1               CUapp            10.0  01/04/2022 00:00:00  2022-04-18   
2                 N/A             8.5                  N/A  2022-04-20   
3               CUapp            10.0  01/04/2022 00:00:00  2022-04-08   

                                     BUNDLE_NAME_y BUNDLE_GROUP_y  \
0                 Combo Bundle CU 10E Subscription       CU COMBO   
1                 Combo Bundle CU 13E Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            N/A   
3                 Combo Bundle CU 10E Subscription       CU COMBO   

  TP_CATEGORY_y  NO_OF_SUBSCRIPTIONS_y  BUNDLE_REVENUE_y ENHANCED_REFERENCE_y  \
0            CU                    1.0              12.0                CUapp   
1            CU                    1.0              13.5                CUapp   
2           N/A                    0.0               0.0                  N/A   
3            CU                    1.0              12.0                CUapp   

  REQUESTING_SYSTEM_y  BUNDLE_PRICE_y  
0               CUapp            12.0  
1               CUapp            13.5  
2                 N/A             8.5  
3               CUapp            12.0

In [196]:
bundles_summary.shape

(574387, 21)

# SHOW MIGRATIONS

In [197]:
bundles_summary.groupby(['BUNDLE_PRICE_x','BUNDLE_PRICE_y']).size()

BUNDLE_PRICE_x  BUNDLE_PRICE_y
8.5             0.0                26931
                8.5                91160
                10.0                6620
                12.0                1890
                13.5                 446
                15.0                 206
                17.5                 227
10.0            0.0                95992
                8.5                 5930
                10.0              241465
                12.0               12549
                13.5               10172
                15.0                5638
                17.5                 280
12.0            0.0                18472
                8.5                 2288
                10.0                9588
                12.0               26250
                13.5                3857
                15.0                2560
                17.5                 379
13.5            0.0                  268
                8.5                    1
                10.0      

In [198]:
migration_categories = bundles_summary.groupby(['BUNDLE_PRICE_x','BUNDLE_PRICE_y']).size()

In [199]:
bundles_summary['HIGHER_BUNDLE'] = 0

In [200]:
conditions = [
    ((bundles_summary["BUNDLE_PRICE_x"] > bundles_summary["BUNDLE_PRICE_y"]) | (bundles_summary["BUNDLE_PRICE_x"] == bundles_summary["BUNDLE_PRICE_y"])),
    (bundles_summary["BUNDLE_PRICE_x"] < bundles_summary["BUNDLE_PRICE_y"])   
    ]

# create a list of the values we want to assign for each condition
values = ["0", "1"]

bundles_summary['HIGHER_BUNDLE'] = np.select(conditions, values)

bundles_summary.head()

MONTH_x DAY_MONTH_x      MSISDN  \
0  01/03/2022 00:00:00  2022-03-07  6900030200   
1  01/03/2022 00:00:00  2022-03-17  6900030559   
2                  N/A  2022-03-24  6900034000   
3  01/03/2022 00:00:00  2022-03-07  6900036262   
4  01/03/2022 00:00:00  2022-03-09  6900039880   

                                     BUNDLE_NAME_x BUNDLE_GROUP_x  \
0   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
1   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            N/A   
3   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   
4   Voice/SMS Bundle CU Combo 500+100 Subscription       CU COMBO   

  TP_CATEGORY_x  NO_OF_SUBSCRIPTIONS_x  BUNDLE_REVENUE_x ENHANCED_REFERENCE_x  \
0            CU                    1.0              10.0                CUapp   
1            CU                    1.0              10.0                CUapp   
2           N/A                    0.0               0.0                  N/A   
3            CU                    1.0              10.0                CUapp   
4            CU                    1.0              10.0                VE407   

  REQUESTING_SYSTEM_x  BUNDLE_PRICE_x              MONTH_y DAY_MONTH_y  \
0               CUapp            10.0  01/04/2022 00:00:00  2022-04-06   
1               CUapp            10.0  01/04/2022 00:00:00  2022-04-18   
2                 N/A             8.5                  N/A  2022-04-20   
3               CUapp            10.0  01/04/2022 00:00:00  2022-04-08   
4              VFShop            10.0                  N/A  2022-04-16   

                                     BUNDLE_NAME_y BUNDLE_GROUP_y  \
0                 Combo Bundle CU 10E Subscription       CU COMBO   
1                 Combo Bundle CU 13E Subscription       CU COMBO   
2  Combo Bundle CU Student Xclusive 7 Subscription            N/A   
3                 Combo Bundle CU 10E Subscription       CU COMBO   
4                                              N/A            N/A   

  TP_CATEGORY_y  NO_OF_SUBSCRIPTIONS_y  BUNDLE_REVENUE_y ENHANCED_REFERENCE_y  \
0            CU                    1.0              12.0                CUapp   
1            CU                    1.0              13.5                CUapp   
2           N/A                    0.0               0.0                  N/A   
3            CU                    1.0              12.0                CUapp   
4           N/A                    0.0               0.0                  N/A   

  REQUESTING_SYSTEM_y  BUNDLE_PRICE_y HIGHER_BUNDLE  
0               CUapp            12.0             1  
1               CUapp            13.5             1  
2                 N/A             8.5             0  
3               CUapp            12.0             1  
4                 N/A             0.0             0

In [201]:
bundles_summary["HIGHER_BUNDLE"].value_counts()

0    529532
1     44855
Name: HIGHER_BUNDLE, dtype: int64

# RENAME - KEEP COLUMNS

In [202]:
#bundles_summary = bundles_summary.rename(mapper = {"BUNDLE_PRICE_x" : "BUNDLE_PRICE", "BUNDLE_PRICE_y" : "BUNDLE_PRICE_NEW"}, axis= 'columns')

In [203]:
bundles_summary = bundles_summary.drop(["BUNDLE_NAME_x", "BUNDLE_PRICE_x", "DAY_MONTH_x", "BUNDLE_GROUP_x", "TP_CATEGORY_x", "NO_OF_SUBSCRIPTIONS_x", "BUNDLE_REVENUE_x", "ENHANCED_REFERENCE_x", "REQUESTING_SYSTEM_x", "MONTH_x",
                                       "BUNDLE_NAME_y", "BUNDLE_PRICE_y", "DAY_MONTH_y", "BUNDLE_GROUP_y", "TP_CATEGORY_y", "NO_OF_SUBSCRIPTIONS_y", "BUNDLE_REVENUE_y", "ENHANCED_REFERENCE_y", "REQUESTING_SYSTEM_y", "MONTH_y"],axis="columns")

In [204]:
bundles_summary.head()

MSISDN HIGHER_BUNDLE
0  6900030200             1
1  6900030559             1
2  6900034000             0
3  6900036262             1
4  6900039880             0

In [205]:
#bundles_summary = bundles_summary.drop(["BUNDLE_NAME_x", "BUNDLE_NAME_y"], axis='columns')

# SAVE

In [206]:
bundles_summary.to_csv(out_dir)